### pandas 라이브러리와 탐색적 데이터 분석 과정 익히기

> 다양한 데이터 분석 케이스를 통해 데이터 분석과 pandas 라이브러리 활용에 대해 익히보기로 합니다.

<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="4em" style="font-weight:bold;color:#3f8dbf;">탐색적 데이터 분석</font><br>

### 전체 판매 프로세스
1. 해당 쇼핑몰에 중소업체가 계약을 맺고
2. 중소업체가 해당 쇼핑몰에 직접 상품을 올리고
2. 고객이 구매하면, 중소업체가 Olist가 제공하는 물류 파트너를 활용해서 배송을 하고,
3. 고객이 상품을 받으면, 고객에게 이메일 survey 가 전송되고,
4. 고객이 이메일 survey 에 별점과 커멘트를 남겨서 제출하게 됨
    
### 데이터 출처
- 브라질에서 가장 큰 백화점의 이커머스 쇼핑몰 (https://olist.com/)
  - 2016년도부터 2018년도 9월까지의 100k 개의 구매 데이터 정보
  - 구매 상태, 가격, 지불수단, 물류 관련, 리뷰관련, 상품 정보, 구매자 지역 관련 정보

### 주요 질문(탐색하고자 하는 질문 리스트)
- 얼마나 많은 고객이 있는가?
- 고객은 어디에 주로 사는가?
- 고객은 주로 어떤 지불방법을 사용하는가?
- 평균 거래액은 얼마일까?
- 일별, 주별, 월별 판매 트렌드는?
- 어떤 카테고리가 가장 많은 상품이 팔렸을까?
- **평균 배송 시간은?**
</div>

### 1. 평균 배송 시간 분석

In [9]:
from google.colab import drive
drive.mount('/gdrive/')


Mounted at /gdrive/


In [10]:
import pandas as pd
PATH = "/gdrive/MyDrive/22년 12월 26일 _컴퓨터_backupfile/주피터_파일/jupyter/taeteacher/0816tae/00_data/"

In [11]:
products = pd.read_csv(PATH + "olist_products_dataset.csv", encoding='utf-8-sig')
customers = pd.read_csv(PATH + "olist_customers_dataset.csv", encoding='utf-8-sig')
geolocation = pd.read_csv(PATH + "olist_geolocation_dataset.csv", encoding='utf-8-sig')
order_items = pd.read_csv(PATH + "olist_order_items_dataset.csv", encoding='utf-8-sig')
payments = pd.read_csv(PATH + "olist_order_payments_dataset.csv", encoding='utf-8-sig')
reviews = pd.read_csv(PATH + "olist_order_reviews_dataset.csv", encoding='utf-8-sig')
orders = pd.read_csv(PATH + "olist_orders_dataset.csv", encoding='utf-8-sig')
sellers = pd.read_csv(PATH + "olist_sellers_dataset.csv", encoding='utf-8-sig')
category_name = pd.read_csv(PATH + "product_category_name_translation.csv", encoding='utf-8-sig')

## psql pgadmin에서 임포트하기위해 pgadmin에 테이블 설계를 위해 컬럼명 확인


In [12]:
print(products.info())
print(customers.info())
print(geolocation.info())
print(order_items.info())
print(payments.info())
print(reviews.info())
print(orders.info())
print(sellers.info())
print(category_name.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    -------------- 

In [ ]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


#### 결측치 갯수 확인하기

In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [ ]:
orders.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

#### 결측치가 있는 행 삭제하기

In [ ]:
orders = orders.dropna()
orders.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96461 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       96461 non-null  object
 1   customer_id                    96461 non-null  object
 2   order_status                   96461 non-null  object
 3   order_purchase_timestamp       96461 non-null  object
 4   order_approved_at              96461 non-null  object
 5   order_delivered_carrier_date   96461 non-null  object
 6   order_delivered_customer_date  96461 non-null  object
 7   order_estimated_delivery_date  96461 non-null  object
dtypes: object(8)
memory usage: 6.6+ MB


#### 배송 시간을 계산해서 새로운 컬럼으로 추가
- 시간 차는 timedelta64 타입으로 표시됨

In [ ]:
orders['delivery_time'] = pd.to_datetime(orders['order_delivered_customer_date']) - pd.to_datetime(orders['order_purchase_timestamp'])

In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96461 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype          
---  ------                         --------------  -----          
 0   order_id                       96461 non-null  object         
 1   customer_id                    96461 non-null  object         
 2   order_status                   96461 non-null  object         
 3   order_purchase_timestamp       96461 non-null  object         
 4   order_approved_at              96461 non-null  object         
 5   order_delivered_carrier_date   96461 non-null  object         
 6   order_delivered_customer_date  96461 non-null  object         
 7   order_estimated_delivery_date  96461 non-null  object         
 8   delivery_time                  96461 non-null  timedelta64[ns]
dtypes: object(8), timedelta64[ns](1)
memory usage: 7.4+ MB


### 2. 통계 활용
- 데이터가 주어졌을 때, 데이터 이해를 위해 가벼운 통계 기법을 사용 
  - 대표적인 예가 평균을 구하는 것임, 이외에 표준편차(std), 최소값(min), 사분위수(25%, 50%, 75%), 최대값(max)

- 데이터에 따라 보다 알맞은 대표값을 구하기 위해, 평균 이외에 다양한 기법이 존재함
  - 가중 평균
    - 데이터값($x_i$) X 가중치($w_i$) 의 총 합을 다시 가중치($w_i$)의 총합으로 나눈 것
    - 예: 여러 기기로부터 가져온 데이터 중, 특정 기기는 신뢰도가 떨어질 경우, 해당 기기로부터 나온 데이터에는 가중치를 낮게 줌
  - 중간값 (가중중간값도 가능)
    - 데이터를 정렬한 후 중간에 위치한 값을 취함
    - 평균은 특잇값(outlier)에 큰 영향을 받으므로, 특잇값에 큰 영향을 받지 않도록 중간값을 활용할 수 있음
  - 절사평균
    - 데이터를 정렬한 후, 양끝에서 일정 개수의 값들을 빼고, 남은 데이터를 기반으로 평균을 계산
    - 즉, 특잇값을 평균을 구할 때 제외하는 것임

In [ ]:
orders['delivery_time'].describe()

count                         96461
mean     12 days 13:23:46.585853350
std       9 days 13:06:45.121880865
min                 0 days 12:48:07
25%                 6 days 18:22:50
50%                10 days 05:11:29
75%                15 days 17:17:10
max               209 days 15:05:12
Name: delivery_time, dtype: object

#### 2.1. boxplot 그래프로 특잇값의 정도 확인해보기

In [ ]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [ ]:
orders['delivery_time'].iplot(kind="box")

#### 2.2. 상위에 있는 특잇값이 문제이므로, 0 ~ 95%에 해당하는 데이터만 사용해서 평균을 계산하기로 함
- pandas에서는 quantile() 함수를 제공
- 입력은 0 ~ 1 사이의 값으로, .95 는 95%에 해당하는 값을 의미

In [ ]:
delivery_time_q95 = orders['delivery_time'].quantile(.95)

In [ ]:
delivery_time_q95

Timedelta('29 days 06:36:33')

In [ ]:
delivery_time_q90 = orders['delivery_time'].quantile(.90)
delivery_time_q90

Timedelta('23 days 02:21:07')

In [ ]:
orders = orders[orders['delivery_time'] < delivery_time_q95]

In [ ]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91637 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype          
---  ------                         --------------  -----          
 0   order_id                       91637 non-null  object         
 1   customer_id                    91637 non-null  object         
 2   order_status                   91637 non-null  object         
 3   order_purchase_timestamp       91637 non-null  object         
 4   order_approved_at              91637 non-null  object         
 5   order_delivered_carrier_date   91637 non-null  object         
 6   order_delivered_customer_date  91637 non-null  object         
 7   order_estimated_delivery_date  91637 non-null  object         
 8   delivery_time                  91637 non-null  timedelta64[ns]
dtypes: object(8), timedelta64[ns](1)
memory usage: 7.0+ MB


### 3. 월별 평균 배송 시간 분석
- orders_date['delivery_time'].dt.total_seconds() : 배송 시간을 초로 변환
  - float 타입으로 변환해서, mean() 평균 계산

In [ ]:
orders_date = orders[['order_purchase_timestamp', 'delivery_time']].copy()
# order_purchase_timestamp 의 날짜 데이터를 기반으로 월별 계산을 해야 하므로 datetime 타입으로 변환
orders_date['order_purchase_timestamp'] = pd.to_datetime(orders_date['order_purchase_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise') 
# delivery_time이 timedelta64 타입인데, 이를 float 타입으로 변환
orders_date['delivery_time'] = orders_date['delivery_time'].dt.total_seconds()#ns를 second로 변환
orders_date = orders_date.set_index('order_purchase_timestamp')

In [ ]:
orders_date.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 91637 entries, 2017-10-02 10:56:33 to 2018-03-08 20:57:30
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   delivery_time  91637 non-null  float64
dtypes: float64(1)
memory usage: 1.4 MB


In [ ]:
orders_date = orders_date.groupby(pd.Grouper(freq='M')).mean() # key 는 기본이 index 임
orders_date.head()

,delivery_time
order_purchase_timestamp,
2016-10-31,1.293386e+06
2016-11-30,NaN
2016-12-31,4.054770e+05
2017-01-31,9.549751e+05
2017-02-28,1.010813e+06


### 일 단위로 배송 시간 변환
- 1일은 86400초

In [ ]:
orders_date['delivery_time'] = orders_date['delivery_time'] / 86400

In [ ]:
orders_date.head()

,delivery_time
order_purchase_timestamp,
2016-10-31,14.969742
2016-11-30,NaN
2016-12-31,4.693021
2017-01-31,11.052953
2017-02-28,11.699220


In [ ]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

In [ ]:
orders_date.iplot(kind="bar")

In [ ]:
orders_date = orders_date[orders_date.index > "2017-01-01"]

- 참고: https://plotly.com/python/reference/

In [ ]:
layout = {
    'xaxis': {
        'showticklabels':True,
        "tick0": "2017-01-31",
        'dtick': 'M1',
        "tickfont": {
            "size": 7                
        }
    }
}
orders_date.iplot(kind="bar", layout=layout)

### plotly 로 세부적인 부분까지 수정해보기

In [ ]:
colors = ['#1B80BF',] * len(orders_date.index)
for index in range(15, len(orders_date.index)):
    colors[index] = '#BF2C47'

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=orders_date.index, 
        y=orders_date['delivery_time'], 
        text=orders_date['delivery_time'], 
        textposition='auto', 
        texttemplate='%{text:.2f} days',
        marker_color=colors
    )
)
fig.update_layout(
    {
        "title": {
            "text": "<b>Average Delivery Time per Month in Brazilian Olist E-Commerce company</b>",
            "x": 0.5,
            "y": 0.9,
            "font": {
                "size": 15
            }
        },
        "xaxis": {
            "title": "from Jan. 2017 to Aug. 2018",
            "showticklabels":True,
            "tick0": "2017-01-31",
            "dtick": "M1",
            "tickfont": {
                "size": 7                
            }
        },
        "yaxis": {
            "title": "Delivery Time (days)"
        },
        "template":'plotly_white'
    }
)

fig.add_annotation(
            x="2018-04-30",
            y=10.4,
            text="<b>Improved logistics</b>",
            showarrow=True,
            font=dict(
                size=10,
                color="#ffffff"
                ),
            align="center",
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#77BDD9",
            ax=40,
            ay=-30,
            bordercolor="#77BDD9",
            borderwidth=2,
            borderpad=4,
            bgcolor="#F22E62",
            opacity=0.8
)


fig.show()